# 3.1 Imports

In [ ]:
# import libraries
import numpy as np
import pandas as pd

import urllib
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

import time
from PIL import Image

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def configure_driver():
    # incognito window
    chrome_options = Options()
    chrome_options.add_argument("--incognito")
    
    # add the argument and make the browser Headless.
    chrome_options.add_argument("--headless")

    # open web browser
    driver = webdriver.Chrome('C:/Users/Leung/Desktop/chromedriver.exe', options=chrome_options)
    return driver

In [ ]:
try:
    WebDriverWait(driver, 5).until(lambda s: s.find_element_by_id("search-results-category-target").is_displayed())
except TimeoutException:
    return None

In [ ]:
# threading lock
# add to the end of the function that is not thread-safe
import threading
csv_write_lock = threading.Lock()

# only one thread allowed
with csv_write_lock:
    with open('data/file_name.csv', mode='a') as f:
        text_writer = csv.writer(f, delimiter=',', quotchar='"', quoting=csv.QUOTE_MINIMAL)
        text_writer.writerow([name, price, img_file, url])

In [ ]:
# END